# Generador de StyleGAN2 ADA en PyTorch

In [ ]:
#@title 1. Monta tu Google Drive
#@markdown Accede a tu Google Drive para cargar modelos pre-entrenados y guardar los resultados.

#@markdown

#@markdown
from google.colab import drive
drive.mount('/content/drive')




In [ ]:
#@title 2. Instalación de librerías
#@markdown StyleGAN2-ADA se instalará en tu Google Drive para acelerar el proceso de entrenamiento.

#@markdown Ejecuta esta celda. Si ya has instalado el repositorio, omitirá el proceso de instalación y actualizará el directorio del repositorio. Si no lo has instalado, instalará todos los archivos necesarios. Notarás algunos errores, ignóralos por ahora ya que son problemas de compatibilidad con Colab que no influyen en nuestro trabajo.
import os
import shlex

!pip install numpy==1.22.4
import numpy as np


if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install ninja opensimplex

%cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- train.py generate.py legacy.py closed_form_factorization.py flesh_digression.py apply_factor.py README.md calc_metrics.py training/stylegan2_multi.py training/training_loop.py util/utilgan.py

# 3. Generación de resultados

In [ ]:
#@title 3.1 Generación de Imágenes
#@markdown Escribe el directorio donde se guardarán las imágenes generadas:
outdir = "/content/drive/MyDrive/ResultadosGAN/coral/net16-2" #@param {type: "string"}
#@markdown Ruta al modelo pre-entrenado. La forma más fácil es buscar el archivo en el directorio de Google Drive que se encuentra en el panel izquierdo, hacer clic en los tres puntos y elegir la opción `copiar ruta`, luego pégalo aquí:
network = "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00070-amonites-11gb-gpu-noaug-resumecustom/network-snapshot-000020.pkl" #@param {type: "string"}
#@markdown ---
#@markdown **Parámetros de Generación**

#@markdown "Truncation", digamos, redondea o trunca los valores del espacio latente. Esto puede tener un efecto sutil o dramático en tus imágenes dependiendo del valor que uses. Cuanto menor sea el número, más realistas deberían aparecer tus imágenes, pero esto también afectará la diversidad. La mayoría de las personas eligen entre 0.5 y 1.0, pero técnicamente es infinito.
truncation =   0.8#@param {type: "number"}
#@markdown Esto te permite elegir semillas aleatorias (puntos de partida) del modelo. Recuerda que nuestra entrada a StyleGAN es una matriz de 512 puntos en tres dimensiones. Estas semillas generarán esos 512x512x512 = 134217728 valores.

#@markdown Cada semilla generará una ruta aleatoria diferente. Pero el mismo valor de semilla siempre generará el mismo conjunto de valores aleatorios, por lo que luego podemos usarlo para otros fines, como la interpolación.
#@markdown
#@markdown Proporciona un número, una lista separada por comas de enteros, o un rango entre valores usando un guión -.
seeds = "1200-1250" #@param {type: "string"}
#@markdown Generación de imágenes aleatorias (cuando se activa el parámetro de *seeds*, se ignorará)
gen_random = True #@param {type: "boolean"}
#@markdown Número de imágenes aleatorias a generar
n_imgs = 20 #@param {type: "integer"}
#@markdown ---
#@markdown **Generación de imágenes no cuadradas**

#@markdown Es posible hacer que el modelo genere imágenes que no sean cuadradas. Esto no es tan bueno como entrenar un modelo rectangular, pero con el modelo adecuado aún puede lucir bien.
gen_nonsquare = False #@param {type: "boolean"}
#@markdown Ancho
width = 640 #@param {type: "integer"}
#@markdown Alto
height = 480 #@param {type: "integer"}
#@markdown Relleno de estilo para aplicar al espacio "extra".
scale_type = 'pad' #@param ['pad', 'padside', 'symm', 'symmside']


if gen_random:
  seeds = ','.join(str(s) for s in list(set(list(np.random.randint(4294967295, size=n_imgs)))))
else:
  seeds = ','.join(str(s).strip() for s in seeds.split(','))
print("Seeds: ", seeds)

nonsquare = f'--size={width}-{height} --scale-type={scale_type}' if gen_nonsquare else ''


outdir = shlex.quote(outdir)
network = shlex.quote(network)
!python generate.py --outdir={outdir} --trunc={truncation} --seeds={seeds} --network={network} {nonsquare}

In [ ]:
#@title 3.2 Image interpolation
#@markdown Escribe el directorio donde se guardarán las imágenes generadas:
outdir = "/content/drive/MyDrive/ResultadosGAN/coral/net16-int2" #@param {type: "string"}
#@markdown Ruta al modelo pre-entrenado. La forma más fácil es buscar el archivo en el directorio de Google Drive que se encuentra en el panel izquierdo, hacer clic en los tres puntos y elegir la opción `copiar ruta`, luego pégalo aquí:
network = "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00070-amonites-11gb-gpu-noaug-resumecustom/network-snapshot-000020.pkl" #@param {type: "string"}
#@markdown ---
#@markdown **Parámetros de Generación**

#@markdown "Truncation", digamos, redondea o trunca los valores del espacio latente. Esto puede tener un efecto sutil o dramático en tus imágenes dependiendo del valor que uses. Cuanto menor sea el número, más realistas deberían aparecer tus imágenes, pero esto también afectará la diversidad. La mayoría de las personas eligen entre 0.5 y 1.0, pero técnicamente es infinito.
truncation =  0.8#@param {type: "number"}

#@markdown Esto te permite elegir semillas aleatorias (puntos de partida) del modelo. Recuerda que nuestra entrada a StyleGAN es una matriz de 512 puntos en tres dimensiones. Estas semillas generarán esos 512x512x512 = 134217728 valores.

#@markdown Cada semilla generará una ruta aleatoria diferente. Pero el mismo valor de semilla siempre generará el mismo conjunto de valores aleatorios, por lo que luego podemos usarlo para otros fines, como la interpolación.

#@markdown Proporciona un número, una lista separada por comas de enteros, o un rango entre valores usando un guión -.
seeds = "632,633,634,634,632" #@param {type: "string"}
#@markdown Generación de imágenes aleatorias (cuando se activa el parámetro de *seeds*, se ignorará)
gen_random = False #@param {type: "boolean"}
#@markdown Número de imágenes aleatorias a generar
n_imgs =  5#@param {type: "integer"}
gen_loop = False
#@markdown ---
#@markdown **Parámetros de Interpolación**
#@markdown
#@markdown Número de fotogramas por imagen
frames = 120 #@param {type: "integer"}

interpolation = 'linear'
#@markdown Espacio Latente
space = 'z' #@param ['z', 'w']
#@markdown ---
#@markdown **Generación de imágenes no cuadradas**

#@markdown Es posible hacer que el modelo genere imágenes que no sean cuadradas. Esto no es tan bueno como entrenar un modelo rectangular, pero con el modelo adecuado aún puede lucir bien.
gen_nonsquare = False #@param {type: "boolean"}
#@markdown Ancho
width = 480 #@param {type: "integer"}
#@markdown Alto
height = 720 #@param {type: "integer"}
#@markdown Relleno de estilo para aplicar al espacio "extra".
scale_type = 'padside' #@param ['pad', 'padside', 'symm', 'symmside']


if gen_random:
  seeds = list(set(list(np.random.randint(4294967295, size=n_imgs))))
  if gen_loop:
    seeds.append(seeds[-1])
  seeds = ','.join(str(s) for s in seeds)
else:
  seeds = ','.join(str(s).strip() for s in seeds.split(','))
print("Seeds: ", seeds)

nonsquare = f'--size={width}-{height} --scale-type={scale_type}' if gen_nonsquare else ''

outdir = shlex.quote(outdir)
network = shlex.quote(network)
!python generate.py --outdir={outdir} --trunc={truncation} --seeds={seeds} --network={network} {nonsquare} --space={space} --process="interpolation" --interpolation={interpolation} --frames={frames}

In [ ]:
#@title 3.3 Truncamiento Transversal

#@markdown En esta celda puedes tomar una semilla (seed) y ver los cambios en ella sobre cualquier cantidad de truncamiento. De -1 a 1 producirán imágenes bastante realistas, pero cuanto más alejado esté del cero, más extrañas se vuelven.

#@markdown Escribe el directorio donde se guardarán las imágenes generadas:
outdir = "/content/drive/MyDrive/ResultadosGAN/coral/net16-trunc4" #@param {type: "string"}
#@markdown Ruta al modelo pre-entrenado. La forma más fácil es buscar el archivo en el directorio de Google Drive que se encuentra en el panel izquierdo, hacer clic en los tres puntos y elegir la opción `copiar ruta`, luego pégalo aquí:
network = "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00000-coral-bleached-ver-11gb-gpu-noaug-resumeffhq512/network-snapshot-000016.pkl" #@param {type: "string"}

#@markdown Elige solo una semilla. Elige una de tus imágenes generadas favoritas:
seed = 939915100  #@param {type: "integer"}
#@markdown Initial truncation value:
start =   0#@param {type: "number"}
#@markdown Valor donde se detiene el truncamiento. Debe ser mayor que el valor inicial: (Se producirá un error si no)
stop = 1.5  #@param {type: "number"}
#@markdown Cuánto debe aumentar el valor de truncamiento en cada fotograma. Establece un valor muy pequeño si deseas una interpolación larga y lenta. (parada - inicio / incremento = fotogramas totales)
increment = 0.005  #@param {type: "number"}

outdir = shlex.quote(outdir)
network = shlex.quote(network)
!python generate.py --process="truncation" --outdir={outdir} --start={start} --stop={stop} --increment={increment} --seeds={seed} --network={network}